# <font size="7"> Datenbeschaffung für Wetterdaten <a class="anchor" id="02-D-01-wetterdaten"></a> </font>

<font size="5">
Die Daten werden von der Internetseite WeatherAPI.com über die History-API bezogen und im Ordner „2-Data Understanding/Datenbeschaffung/Wetter/“ gespeichert. Es werden die Daten vom 01.01.2015 bis zum 31.12.2021 verwendet. 
<br>Die Dokumentation der API ist unter folgendem Link erreichbar: https://www.weatherapi.com/docs/
</font>

In [1]:
import pandas as pd

#Daten als Index vom 01.01.2015 bis 31.12.2021
indices = pd.date_range('01/01/2015', periods=2557, freq='D')
#indices = pd.date_range('11/29/2018', periods=2, freq='D')

print("Von\t", indices[0])
print("Bis\t", indices[len(indices)-1])

#Städenamen und Koordinaten festlegen
cities = [
    ("Freiburg","48,7.85"),
    ("Mannheim","49.49,8.46"),
    ("Stuttgart","48.77,9.18"),
    ("Ulm","48.4010822,9.9876076")]

#city = "Freiburg"; coordinates = "48,7.85"
#city = "Mannheim"; coordinates = "49.49,8.46"
#city = "Stuttgart"; coordinates = "48.77,9.18"
#city = "Ulm"; coordinates = "48.4010822,9.9876076"

Von	 2015-01-01 00:00:00
Bis	 2021-12-31 00:00:00


<font size="5">
Für jede Stadt werden die Wetterdaten eines Tages mittels Aufrufes der API im JSON-Format heruntergeladen und im jeweiligen Ordner der Stadt gespeichert. Als Name wird das jeweilige Datum verwendet.
</font>

In [ ]:
import pandas as pd
import requests

#Schleife, um Daten für alle Städte seriell zu laden
for city, coordinates in cities:

    print(city, "\t", coordinates)

    #Durchlaufen aller Tage
    for index in indices:

        #URL für Request bei API
        url = "https://weatherapi-com.p.rapidapi.com/history.json"

        #Datum für API formatieren
        date = index.date().strftime(format="%Y-%m-%d")

        #Abfrage für API
        request = {"q":coordinates,"dt":date,"lang":"en"}

        #Header für API
        header = {
            'x-rapidapi-key': "",
            'x-rapidapi-host': "weatherapi-com.p.rapidapi.com"
            }

        #API aufrufen und JSON speichern
        response = requests.request("GET", url, headers=header, params=request)

        #JSON-Datei im Ordner für jeweilige Stadt speichern
        file = open(("Wetter/" + city + "/" + date + ".json"), "w")
        file.write(response.text)
        file.close()

        print(index.date())

<font size="5">
Für jede Stadt werden die JSON-Dateien mit den jeweiligen Wetterdaten für jeden Tag seriell eingelesen. Es werden dann die relevanten Attribute aus der JSON-Datei geladen und an einen DataFrame angehangen. Als Index wird das jeweilige Datum verwendet. Viele Daten sind in mehreren Einheiten angegeben, Temperaturen sind beispielsweise in Grad Celsius und Fahrenheit angegeben, Geschwindigkeiten in Kilometer und Meilen pro Stunde etc. Es werden allerdings nur Einheiten aus dem metrischen System und in Grad Celsius verwendet, alle übrigen Attribute werden entfernt. Auch werden Daten wie beispielsweise der Mondauf- und Monduntergang entfernt, da diese Daten nicht Gegenstand der Analyse sind. Abschließend werden die Wetterdaten für die jeweilige Stadt in einer CSV-Datei im Ordner „2-Data Understanding/Datenbeschaffung/“ gespeichert.
</font>

In [ ]:
import pandas as pd
pd.set_option('display.expand_frame_repr', False)
import json

#Schleife, um Daten für alle Städte seriell zu verarbeiten
for city, coordinates in cities:
    
    print(city, "\t", coordinates)
    
    #DataFrame für Wetterdaten erstellen
    df_weather = pd.DataFrame()
    
    #Durchlaufen aller Tage
    for index in indices:
        
        #JSON-Datei für jeweiligen Tag laden
        file_path = "Wetter/" + city + "/" + index.date().strftime(format="%Y-%m-%d") + ".json"
        file = open(file_path, "r")
        
        #JSON-String auslesen
        content = json.loads(file.read())

        #Wetterdaten für jeweiligen Tag in Zeile des DataFrames umwandeln
        df_day = pd.DataFrame.from_dict(content["forecast"]["forecastday"][0]["day"])
        df_day = df_day.iloc[2]
        
        #Astrodaten anhängen
        df_astro = pd.DataFrame.from_dict(content["forecast"]["forecastday"][0]["astro"], orient="index").transpose()
        df_day = df_day.append(df_astro.squeeze())

        #Zeile für jeweiligen Tag an DataFrame anhängen
        df_weather = df_weather.append(df_day, ignore_index=True)

        print(index.date())

    #Index setzen
    df_weather.set_index(indices, inplace=True)
    df_weather.index.name = "datum"
    
    #Wetterdaten bereinigen (Daten in Fahren/Meilen entfernen, Daten zum Mond entfernen)
    df_weather.drop(["avgtemp_f", "maxtemp_f", "mintemp_f", "avgvis_miles", "maxwind_mph", "totalprecip_in", "uv", "moonrise", "moonset", "moon_phase", "moon_illumination"], axis=1, inplace=True)
    #Spalten umbenennen
    df_weather.columns = ["luftfeuchtigkeit", "durchschnittstemperatur", "sichtweite", "wetter", "hoechsttemperatur", "windgeschwindigkeit", "tiefsttemperatur", "sonnenaufgang", "sonnenuntergang", "niederschlag"]
    
    #Wetterdaten für jeweilige Stadt speichern
    filename = (city + ".csv").lower()
    df_weather.to_csv(filename)